In [1]:
import os
%pwd

'd:\\Deep Learning\\End to End NLP project\\text-summarizer-app\\research'

In [2]:
os.chdir("../")
%pwd

'd:\\Deep Learning\\End to End NLP project\\text-summarizer-app'

In [3]:
#entity file definition to define the return type of a function
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    ALL_REQUIRED_FILES: list                                                                                                           

In [4]:
#src/config/configuration.py definition to centralize the project settings, paths, and parameters. 
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            ALL_REQUIRED_FILES=config.ALL_REQUIRED_FILES 
        )

        return data_validation_config

In [5]:
#components file definition
import os
from textSummarizer.logging import logger

class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config


    
    def validate_all_files_exist(self)-> bool:
        try:
            validation_status = None

            all_files = os.listdir(os.path.join("artifacts","data_ingestion","samsum_dataset"))

            for file in all_files:
                if file not in self.config.ALL_REQUIRED_FILES:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                    logger.error(f"File {file} not found in required files: {self.config.ALL_REQUIRED_FILES}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                    logger.info(f"File {file} found in required files: {self.config.ALL_REQUIRED_FILES}")
            return validation_status
        
        except Exception as e:
            raise e

In [6]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_files_exist()
except Exception as e:
    raise e

[2024-12-06 12:41:13,451: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-06 12:41:13,457: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-06 12:41:13,461: INFO: common: created directory at: artifacts]
[2024-12-06 12:41:13,465: INFO: common: created directory at: artifacts/data_validation]
[2024-12-06 12:41:13,470: ERROR: 2377377019: File dataset_dict.json not found in required files: ['train', 'test', 'validation']]
[2024-12-06 12:41:13,474: INFO: 2377377019: File test found in required files: ['train', 'test', 'validation']]
[2024-12-06 12:41:13,480: INFO: 2377377019: File train found in required files: ['train', 'test', 'validation']]
[2024-12-06 12:41:13,484: INFO: 2377377019: File validation found in required files: ['train', 'test', 'validation']]
